# Multimodal RAG v1

In [1]:
from langchain.text_splitter import CharacterTextSplitter
from unstructured.partition.pdf import partition_pdf

C:\Users\Anastasia\sber-reports-rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Функция извлечения элементов из pdf-файла
def extract_pdf_elements(path, fname):
    """
    Функция для извлечения различных элементов из PDF-файла, таких как изображения, таблицы,
    и текста. Также осуществляется разбиение текста на части (чанки) для дальнейшей обработки.

    Аргументы:
    path: Строка, содержащая путь к директории, в которую будут сохранены извлеченные изображения.
    fname: Строка, содержащая имя PDF-файла, который необходимо обработать.

    Возвращает:
    Список объектов типа `unstructured.documents.elements`, представляющих извлеченные из PDF элементы.
    """
    return partition_pdf(
        filename=path + fname,          # Путь к файлу, который нужно обработать
        extract_images_in_pdf=True,     # Указание на то, что из PDF нужно извлечь изображения
        infer_table_structure=True,     # Автоматическое определение структуры таблиц в документе
        strategy="hi_res", # Ваш код здесь,   # Стратегия разбиения текста на части
        max_characters=1024,# Ваш код здесь,            # Максимальное количество символов в одном чанке текста
        new_after_n_chars=1024, # Ваш код здесь,         # Число символов, после которого начинается новый чанк текста
        combine_text_under_n_chars=512,# Ваш код здесь,# Минимальное количество символов, при котором чанки объединяются
        image_output_dir_path=path,     # Путь, куда будут сохраняться извлеченные изображения
    )


# Функция категоризации элементов
def categorize_elements(raw_pdf_elements):
    """
    Функция для категоризации извлеченных элементов из PDF-файла.
    Элементы делятся на текстовые элементы и таблицы.

    Аргументы:
    raw_pdf_elements: Список объектов типа `unstructured.documents.elements`,
                      представляющих извлеченные из PDF элементы.

    Возвращает:
    Два списка: texts (текстовые элементы) и tables (таблицы).
    """
    tables = []  # Список для хранения элементов типа "таблица"
    texts = []   # Список для хранения текстовых элементов
    for element in raw_pdf_elements:
        # Проверка типа элемента. Если элемент является таблицей, добавляем его в список таблиц
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        # Если элемент является композитным текстовым элементом, добавляем его в список текстов
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)): # .Text
            texts.append(str(element))
    return texts, tables  # Возвращаем списки с текстами и таблицами


# Указываем путь до директории, где находится PDF-файл
fpath = "../data/raw/"
# Указываем имя PDF-файла, который нужно обработать
fname = "sber10.pdf"

# Извлекаем элементы из PDF-файла с помощью функции extract_pdf_elements
raw_pdf_elements = extract_pdf_elements(fpath, fname)

# Категоризируем извлеченные элементы на текстовые и табличные с помощью функции categorize_elements
texts, tables = categorize_elements(raw_pdf_elements)

# Создаем объект CharacterTextSplitter для разбиения текста на части (чанки)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, # Максимальный размер чанка в символах
    chunk_overlap=0 # Количество перекрывающихся символов между чанками
)

# Объединяем все текстовые элементы в одну строку
joined_texts = " ".join(texts)

# Разбиваем объединенный текст на чанки, используя созданный CharacterTextSplitter
texts_4k_token = text_splitter.split_text(joined_texts)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
tables

['Отчет по рискам 11 Сбер сегодня 104 Отчет по рискам 12 Культура выполнения ключевых стратегических целей 13 Миссия и ценности 14 Экосистема В2С в 2023 году 15 Экосистема В2В в 2023 году Корпоративное 118 Система корпоративного управления 16 Сеть продаж в 2023 году 124 Общее собрание акционеров управление 17 Технологическое лидерство в 2023 году 126 Наблюдательный совет 18 Искусственный интеллект в 2023 году 134 Комитеты Наблюдательного совета 19 Управление рисками в 2023 году 140 Исполнительные органы. Правление 20 Ключевые финансовые результаты 141 Корпоративный секретарь 21 Долгосрочное видение и Стратегия 2026 141 Система контроля и аудита 144 Комплаенс и внутренний контроль в Сбере в 2023 году 147 Кредитные рейтинги Обзор рынка 147 Акционерный капитал 27 Экосистема В2С 151 Взаимодействие с инвесторами и акционерами 58 Экосистема В2В 155 Крупные сделки 76 Сеть продаж 155 Сделки с заинтере сован ностью 82 Технологическое лидерство 156 Инвестиционная привлекательность Приложения 94 

In [7]:
texts

[]

In [13]:
raw_pdf_elements[2]

In [12]:
raw_pdf_elements[2].text

'3 В Отчете освещаются события за 2023 календарный год, за исключением случаев, прямо указанных по тексту.'